# Multivariable Functions: Domain and Range with SymPy

In this notebook we study the function
$$
z = f(x,y) = \sqrt{y - x^2}
$$
as a **real-valued function of two variables**.

Our goals are:

1. Determine the **domain** of $f$ (as a subset of $\mathbb{R}^2$).
2. Determine the **range** of $f$ (as a subset of $\mathbb{R}$).
3. Represent the domain and range using SymPy objects.
4. Visualize the domain region and the graph of $z=f(x,y)$.
5. Practice encoding other examples using the same pattern.


In [ ]:
# 1. Basic setup: import SymPy and configure printing

import sympy as sp
sp.init_printing(use_latex="mathjax")

# Define symbols
x, y, w = sp.symbols('x y w', real=True)
x: sp.Symbol = sp.Symbol('x', real=True)
y: sp.Symbol = sp.Symbol('y', real=True)
w: sp.Symbol = sp.Symbol('w', real=True)
assert isinstance(x, sp.Symbol)
assert isinstance(y, sp.Symbol)
assert isinstance(w, sp.Symbol)

# Define the function z = f(x,y)
f: sp.Expr = sp.sqrt(y - x**2)
assert isinstance(f, sp.Expr)
f


<class 'sympy.core.power.Pow'>


   __________
  ╱    2     
╲╱  - x  + y 